block stream by event.  
f = a + d and e = f + c may run concurrently.  

0 2 4 6 8 10 12 14 16 18 20 22 24 26 28 30 32 34 36 38 40 42 44 46 48 50 52 54 56 58 60 62 64 66 68 70 72 74 76 78 80 82 84 86 88 90 92 94 96 98 100 102 104 106 108 110 112 114 116 118 120 122 124 126 128 130 132 134 136 138 140 142 144 146 148 150 152 154 156 158 160 162 164 166 168 170 172 174 176 178 180 182 184 186 188 190 192 194 196 198 200 202 204 206 208 210 212 214 216 218 220 222 224 226 228 230 232 234 236 238 240 242 244 246 248 250 252 254
256 258 260 262 264 266 268 270 272 274 276 278 280 282 284 286 288 290 292 294 296 298 300 302 304 306 308 310 312 314 316 318 320 322 324 326 328 330 332 334 336 338 340 342 344 346 348 350 352 354 356 358 360 362 364 366 368 370 372 374 376 378 380 382 384 386 388 390 392 394 396 398 400 402 404 406 408 410 412 414 416 418 420 422 424 426 428 430 432 434 436 438 440 442 444 446 448 450 452 454 456 458 460 462 464 466 468 470 472 474 476 478 480 482 484 486 488 490 492 494 496 498 500 502 504 506 508 510
...

1 7 13 19 25 31 37 43 49 55 61 67 73 79 85 91 97 103 109 115 121 127 133 139 145 151 157 163 169 175 181 187 193 199 205 211 217 223 229 235 241 247 253 259 265 271 277 283 289 295 301 307 313 319 325 331 337 343 349 355 361 367 373 379 385 391 397 403 409 415 421 427 433 439 445 451 457 463 469 475 481 487 493 499 505 511 517 523 529 535 541 547 553 559 565 571 577 583 589 595 601 607 613 619 625 631 637 643 649 655 661 667 673 679 685 691 697 703 709 715 721 727 733 739 745 751 757 763
769 775 781 787 793 799 805 811 817 823 829 835 841 847 853 859 865 871 877 883 889 895 901 907 913 919 925 931 937 943 949 955 961 967 973 979 985 991 997 1003 1009 1015 1021 1027 1033 1039 1045 1051 1057 1063 1069 1075 1081 1087 1093 1099 1105 1111 1117 1123 1129 1135 1141 1147 1153 1159 1165 1171 1177 1183 1189 1195 1201 1207 1213 1219 1225 1231 1237 1243 1249 1255 1261 1267 1273 1279 1285 1291 1297 1303 1309 1315 1321 1327 1333 1339 1345 1351 1357 1363 1369 1375 1381 1387 1393 1399 1405 1411 1417 1423 1429 1435 1441 1447 1453 1459 1465 1471 1477 1483 1489 1495 1501 1507 1513 1519 1525 1531
...

In [ ]:
%%writefile stream_event.cu
#include <stdio.h>
#include <stdlib.h>
#include <type_traits>
#include <iostream>
#include <cuda_runtime.h>
#include <chrono>

#define TYPE int
#define N 512


template <typename T, typename = std::enable_if_t<std::is_arithmetic<T>::value>>
__global__ void device_add(T *a, T *b, T *c, int n)
{
    int id = threadIdx.x + blockIdx.x * blockDim.x;
    if (id < n) {
        c[id] = a[id] + b[id];
    }
}

template <typename T, typename = std::enable_if_t<std::is_arithmetic<T>::value>>
__global__ void device_add_one(T *a, int m, int n)
{
    int id = threadIdx.x + blockIdx.x * blockDim.x;
    if (id < n) {
        a[id] = a[id] + m;
    }
}

template <typename T, typename = std::enable_if_t<std::is_arithmetic<T>::value>>
__global__ void device_multiple(T *a, int m, int n)
{
    int id = threadIdx.x + blockIdx.x * blockDim.x;
    if (id < n) {
        a[id] = a[id] * m;
        __nanosleep(1000000ULL);
    }
}

template <typename T, typename = std::enable_if_t<std::is_arithmetic<T>::value>>
void print_output(T *a, T *b, T *c)
{
    for (int i = 0; i < N; ++i)
    {
        printf("\n %d + %d  = %d", a[i], b[i], c[i]);
    }
}

template <typename T, typename = std::enable_if_t<std::is_arithmetic<T>::value>>
void print_output(T *a)
{
    for (int i = 0; i < N; ++i)
    {
        if (i % 128 == 0)
          printf("\n");
        printf("%d ", a[i]);
    }
    printf("\n");
}

int main()
{
    TYPE *a, *b, *c, *d, *e, *f;
    TYPE *d_a, *d_b, *d_c, *d_d, *d_e, *d_f;
    int threads_per_block = 0, no_of_blocks = 0;

    int size = N * sizeof(TYPE);

    // Allocate space for host copies of a, b, c, d and setup input values
    cudaHostAlloc(&a, size, cudaHostAllocDefault);
    cudaHostAlloc(&b, size, cudaHostAllocDefault);
    cudaHostAlloc(&c, size, cudaHostAllocDefault);
    cudaHostAlloc(&d, size, cudaHostAllocDefault);
    cudaHostAlloc(&e, size, cudaHostAllocDefault);
    cudaHostAlloc(&f, size, cudaHostAllocDefault);

    for (int i = 0; i < N; ++i)
    {
        a[i] = i;
        b[i] = i;
        c[i] = i;
        d[i] = i;
    }

    // Allocate space for device copies of a, b, c, d
    cudaMalloc((void **)&d_a, size);
    cudaMalloc((void **)&d_b, size);
    cudaMalloc((void **)&d_c, size);
    cudaMalloc((void **)&d_d, size);
    cudaMalloc((void **)&d_e, size);
    cudaMalloc((void **)&d_f, size);

    int leastPriority, greatestPriority;
    cudaDeviceGetStreamPriorityRange(&leastPriority, &greatestPriority);
    std::cout<<"leastPriority:"<<leastPriority<<" "<<"greatestPriority:"<<greatestPriority<<std::endl;

    // Create n streams
    cudaStream_t stream0, stream1;
    cudaEvent_t event0, event1;
    // cudaStreamCreate(&stream0);
    // cudaStreamCreate(&stream1);
    cudaStreamCreateWithPriority(&stream0, cudaStreamNonBlocking, leastPriority);
    cudaStreamCreateWithPriority(&stream1, cudaStreamNonBlocking, greatestPriority);
    cudaEventCreate(&event0);
    cudaEventCreate(&event1);

    // Copy data from host to device
    cudaMemcpyAsync(d_a, a, size, cudaMemcpyHostToDevice, stream0);
    cudaMemcpyAsync(d_b, b, size, cudaMemcpyHostToDevice, stream1);
    cudaMemcpyAsync(d_c, c, size, cudaMemcpyHostToDevice, stream1);
    cudaMemcpyAsync(d_d, d, size, cudaMemcpyHostToDevice, stream0);

    // Execute kernel
    threads_per_block = 128;
    no_of_blocks = (N + threads_per_block - 1) / threads_per_block;

    // a = a * 2 stream0
    // b = b * 3 stream1
    // c = c * 4 stream1
    // d = d * 5 stream0
    device_multiple<<<no_of_blocks, threads_per_block, 0, stream0>>>(d_a, 2, N);
    device_multiple<<<no_of_blocks, threads_per_block, 0, stream1>>>(d_b, 2, N);
    device_multiple<<<no_of_blocks, threads_per_block, 0, stream1>>>(d_c, 2, N);
    device_multiple<<<no_of_blocks, threads_per_block, 0, stream0>>>(d_d, 2, N);

    device_add_one<<<no_of_blocks, threads_per_block, 0, stream1>>>(d_d, 1, N);

    // f = a + d stream1
    // cudaEventRecord(event0, stream0);
    // cudaStreamWaitEvent(stream1, event0, 0);
    device_add<<<no_of_blocks, threads_per_block, 0, stream1>>>(d_a, d_d, d_f, N);

    // e = f + c stream0
    // device_add<<<no_of_blocks, threads_per_block, 0, stream0>>>(d_e, d_b, d_c, N);
    cudaEventRecord(event1, stream1);
    cudaStreamWaitEvent(stream0, event1, 0);
    device_add<<<no_of_blocks, threads_per_block, 0, stream0>>>(d_f, d_c, d_e, N);

    // Copy result back to host
    cudaMemcpyAsync(a, d_a, size, cudaMemcpyDeviceToHost, stream0);
    cudaMemcpyAsync(d, d_d, size, cudaMemcpyDeviceToHost, stream0);
    cudaMemcpyAsync(e, d_e, size, cudaMemcpyDeviceToHost, stream0);
    cudaMemcpyAsync(f, d_f, size, cudaMemcpyDeviceToHost, stream1);

    cudaDeviceSynchronize();

    // print_output(a, b, c);
    print_output(a);
    print_output(d);
    print_output(f);

    print_output(e);

    // Clean up
    cudaStreamDestroy(stream0);
    cudaStreamDestroy(stream1);
    cudaEventDestroy(event0);
    cudaEventDestroy(event1);

    cudaFreeHost(a);
    cudaFreeHost(b);
    cudaFreeHost(c);
    cudaFreeHost(d);
    cudaFreeHost(e);
    cudaFreeHost(f);
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    cudaFree(d_d);
    cudaFree(d_e);
    cudaFree(d_f);

    return 0;
}

In [ ]:
!nvcc -o stream_event -lineinfo -arch=sm_70 stream_event.cu

In [ ]:
!./stream_event

In [ ]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/nsight-systems-2024.2.3_2024.2.3.38-1_amd64.deb
!apt update
!apt install ./nsight-systems-2024.2.3_2024.2.3.38-1_amd64.deb
!apt --fix-broken install

In [ ]:
!nsys profile -o report_nsys_stream_event -f true ./stream_event